In [23]:
# Importação dos modulos necessários:
import nest_asyncio
nest_asyncio.apply()
import asyncio
from multiprocessing import Pool, cpu_count
import concurrent.futures
import sys
import time
from enum import Enum
import aiohttp
import requests

In [24]:
# Criando o 'get_ids' da cervejaria:
def get_ids_cervejarias():
    r = requests.get('https://api.openbrewerydb.org/breweries')
    if r.status_code == 200:
        return [r['id'] for r in r.json()]

In [25]:
# Coletando os 'ids' da cervejaria:
def get_cervejaria(id_c):
    r = requests.get(f'https://api.openbrewerydb.org/breweries/{id_c}')
    if r.status_code == 200:
        print(r.json())

In [7]:
# Requerendo e coletando os dados via 'async':
async def get_cervejaria_async(id_c, session):
    async with session.get(f'https://api.openbrewerydb.org/breweries/{id_c}') as response:
        print(await response.json())


In [10]:
# Fazendo o processamento dos dados via 'aiohttp':
async def processar_async(ids):
    coros = []
    async with aiohttp.ClientSession() as session:
        for id_c in ids:
            coros.append(get_cervejaria_async(id_c, session))
        await asyncio.gather(*coros)


In [12]:
# Criando o get da cervejaria de forma manual:
async def get_cervejaria_async_manual(id_c):
    loop = asyncio.get_running_loop()
    return await loop.run_in_executor(None, get_cervejaria, id_c)

In [16]:
# Criando os tipos de execução para cada processo:
class TipoExecucao(Enum):
    UM_PROCESSO = 1
    VARIAS_THREADS = 2
    ASYNCIO_COM_LIB_HTTP = 3


In [ ]:
# Iniciando as execuções:
if __name__ == '__main__':
    started = time.time()

    tipo_execucao = TipoExecucao.ASYNCIO_COM_LIB_HTTP
    cores = cpu_count()

    ids_cervejarias = get_ids_cervejarias()

    if TipoExecucao.UM_PROCESSO == tipo_execucao:
        # Usando somente um processo
        for id_cervejaria in ids_cervejarias:
            get_cervejaria(id_cervejaria)

    elif TipoExecucao.VARIAS_THREADS == tipo_execucao:
        # Usando várias threads
        with concurrent.futures.ThreadPoolExecutor(cores) as thp:
            thp.map(get_cervejaria, ids_cervejarias)
    
    elif TipoExecucao.ASYNCIO_COM_LIB_HTTP == tipo_execucao:
        # Usando o asyncio
        event_loop = asyncio.get_event_loop()
        task = []
        event_loop.run_until_complete(processar_async(ids_cervejarias))

    elif TipoExecucao.ASCYNCIO_MANUAL == tipo_execucao:
        # Usando asyncio manual
        event_loop = asyncio.get_event_loop()
        tasks = []

        for id_cervejaria in ids_cervejarias:
            tasks.append(get_cervejaria_async_manual(id_cervejaria))
        event_loop.run_until_complete(asyncio.wait(tasks))
    elapsed = time.time()
    print('Time taken :', elapsed - started)